# Cut Labels ("Custom User Time" Labels?)

Cut Labels in LUSID are used to simplify timestamps and streamline the usage of LUSID in a global scenario across multiple timezones.

Instead of providing a date, time, and time zone in every transaction, LUSID can be provided with a date and a Cut Label - a user-defined tag describing a timezone and a specific local time in that time zone.

For example:

- UTC offset time: 2018-08-31T07:36:53.523415+01:00
- Cut label, assuming SingaporeClose has been configured: 2018-08-31NSingaporeClose

Find out more about Cut Labels in LUSID here:
[LUSID Knowledge Base: Cut Labels](https://support.lusid.com/what-are-cutlabels)

This example will cover how to set up and customise Cut Labels, and how to use these in LUSID. 

This example will outline how to:

1. Setting Up Cut Labels
    - 1.1. Creating Cut Labels
    - 1.2. Cut Label Details
    - 1.3. Deleting a Cut Label
2. Create a Transaction Portfolio
3. Updating our Portfolio
4. Add your Instrument Universe
5. Set your Initial Holdings
6. Get Output Transactions
7. Add your Transactions Throughout the Day
8. Retrieve your Transactions
9. Retrieve your Holdings
10. Adjust Holdings with your End of Day Position
11. Cancelling Holdings Adjustments
12. Delete Cut Labels and Portfolios

*First, run the cell below to import libraries and create the LUSID client to initialise our environment*

In [1]:
# Import LUSID
import lusid
import lusid.models as models
import lusid_sample_data as import_data
# Import Libraries
import pprint
from datetime import datetime, timedelta, time, date
import pytz
import uuid
import printer as prettyprint
from datetime import datetime
import pandas as pd
import numpy as np
import os
import json

# Authenticate our user and create our API client
client = import_data.authenticate_secrets()

print('LUSID Environment Initialised')
print('LUSID version : ', client.metadata.get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID version :  0.5.3057.0


### Initial Setup
The cell below contains a number of helper functions used throughout this notebook - these are not necessary for any of the methods demonstrated, but simplify the readability and usability of this module

In [2]:
# The following function creates a random alphanumeric code of 4 characters that can be appended to Ids
# and Names to ensure they remain unique throughout multiple runs of this example
def get_guid():
    return str(uuid.uuid4())[:4]

Later in this example we will create some dummy cut labels in order to demonstrate their use.

*Run the cell below to load a csv of cut label data*

In [3]:
# Load CSV file to create more cut labels from
data = pd.read_csv("data/dummy_cut_labels.csv")
data.head()

,display_name,hours,minutes,time_zone
0,cut-label-A,1,0,GB
1,cut-label-B,2,0,GB
2,cut-label-C,3,0,GB
3,cut-label-D,4,0,GB


## 1. Setting Up Cut Labels

### 1.1. Creating Cut Labels

We begin by creating a single cut label, named *"LondonClose"*, by defining the parameters of the cut label and upserting this into LUSID.

*Run the cell below to create a cut label and print its information*

In [4]:
# Create the time for the cut label
londonTime1 = models.CutLocalTime(hours=17, minutes=0)

# Define the parameters of the cut label in a request
request = models.CutLabelDefinition(
    code="LondonClose", 
    description="London Closing Time, 5pm in UK", 
    display_name="LondonClose",
    cut_local_time=londonTime1,
    time_zone="GB")

# Upsert the request to LUSID to create the cut label
try: 
    result = client.cut_labels.create_cut_label_definition(
       create_request=request)
except lusid.ApiException as error: 
    print("error: " + json.loads(error.body)["detail"] + "\n")
    result = client.cut_labels.get_cut_label_definition(
        code="LondonClose")

prettyprint.cut_label_response(result)

Cut Label Created
Display Name: LondonClose
Code: LondonClose
Local Time: 17:00
Timezone: GB
Description: London Closing Time, 5pm in UK



Now we will create multiple more cut labels from the csv data by the same method, and then list the details of all of our existing cut labels

*Run the cell below to create cut labels and print their information*

In [5]:
# Create cut labels from the loaded CSV data
for index, row in data.iterrows():
    print("Creating Cut Label ", index)
    guid = get_guid()
    # Create a request for our cut label
    request = models.CutLabelDefinition(
        code=row["display_name"]+"-"+guid,
        display_name=row["display_name"],
        description="",
        cut_local_time=models.CutLocalTime(
            hours=row["hours"], 
            minutes=row["minutes"]),
        time_zone=row["time_zone"])
    # Upsert the request to LUSID to create the cut label
    result = client.cut_labels.create_cut_label_definition(
        create_request=request)
    print("...")

print("")

# List the details of all existing cut labels
response = client.cut_labels.list_cut_label_definitions()
prettyprint.list_cut_label_details(response)

Creating Cut Label  0
...
Creating Cut Label  1
...
Creating Cut Label  2
...
Creating Cut Label  3
...

Existing Cut Labels:
Display Name: cut-label-C
Code: cut-label-C-b639
Local Time: 3:00
Timezone: GB
Description: 

Display Name: LondonClose
Code: LondonClose
Local Time: 17:00
Timezone: GB
Description: London Closing Time, 5pm in UK

Display Name: cut-label-B
Code: cut-label-B-c942
Local Time: 2:00
Timezone: GB
Description: 

Display Name: cut-label-D
Code: cut-label-D-5a71
Local Time: 4:00
Timezone: GB
Description: 

Display Name: cut-label-A
Code: cut-label-A-0bf2
Local Time: 1:00
Timezone: GB
Description: 



### 1.2 Cut Label Details

#### 1.2.1 Get Cut Label Details
*Run the cell below to get details on the cut label "LondonClose"*

In [6]:
# First list the original details for the cut label we are going to change
details = client.cut_labels.get_cut_label_definition(
    code="LondonClose")
prettyprint.get_cut_label(details)

Cut Label Details:
Display Name: LondonClose
Code: LondonClose
Local Time: 17:00
Timezone: GB
Description: London Closing Time, 5pm in UK



#### 1.2.2. Update a Cut Label
Now we can modify the cut label's display name, description, time, and timezone

*Run the cell below to modify the details of the cut label*

In [7]:
# Modify London Close cut label time to 5:30pm
londonTime2 = models.CutLocalTime(hours=17, minutes=30)

request = models.CutLabelDefinition(
    description="London Closing Time, 5:30pm in UK", 
    display_name="LondonClose",
    cut_local_time=londonTime2,
    time_zone="GB")

response = client.cut_labels.update_cut_label_definition(
    code="LondonClose",
    update_request=request)

# Print the updated London Close cut label
prettyprint.update_cut_label(response)

Updated Cut Label:
Display Name: LondonClose
Code: LondonClose
Local Time: 17:30
Timezone: GB
Description: London Closing Time, 5:30pm in UK



### 1.3. Deleting a Cut Label
Now we can delete cut labels that are no longer required. First we will create a dummy cut label to be deleted, and then list out the display names of all our cut labels.  

*Run the cell below to create a cut label*

In [8]:
# Add a dummy Cut Label to be deleted:
# First set our time
midnight = models.CutLocalTime(hours=0, minutes=0)

# Then create a request for our cut label
request = models.CutLabelDefinition(
    code="DeleteMe", 
    description="Dummy Cut Label to be deleted", 
    display_name="DeleteMe",
    cut_local_time=midnight,
    time_zone="GB")

# Upsert the request to LUSID to create the cut label
result = client.cut_labels.create_cut_label_definition(
   create_request=request)

# Shows all cut label display names
prettyprint.list_cut_labels(client.cut_labels.list_cut_label_definitions())

Existing Cut Labels:
DeleteMe
cut-label-C
cut-label-B
LondonClose
cut-label-D
cut-label-A


Next, we will delete our dummy cut label and list out the display names of our cut labels again - DeleteMe has been deleted!  

*Run the cell below to delete the cut label*

In [9]:
# Delete cut label "Delete Me"
client.cut_labels.delete_cut_label_definition(
    code="DeleteMe")

# Shows all cut label display names
prettyprint.list_cut_labels(client.cut_labels.list_cut_label_definitions())

Existing Cut Labels:
cut-label-C
LondonClose
cut-label-B
cut-label-D
cut-label-A


## 2. Create a Transaction Portfolio
We are now going to look at some example use cases for our cut labels. To do this we are going to need to create a portfolio, and in order to create a portfolio you will need to create a scope. Read more about scopes in the [LUSID Knowledge Base: Scopes](https://support.lusid.com/what-is-a-scope-in-lusid-and-how-is-it-used).

Once you have a scope, you can create a portfolio inside this scope. Every portfolio can be referenced by a unique code. Read more about portfolios in the [LUSID Knowledge Base: Portfolios](https://support.lusid.com/what-is-2).

For further usage of the create portfolio API call refer to the [LUSID API Docs: Create Portfolio](https://docs.lusid.com/#operation/CreatePortfolio).

> Note that currently the create portfolio API call does not support cut labels - make sure to check the api docs for *"date-time"* or *"dateorcutlabel"* for each parameter type

*Run the cell below to create your scope and portfolio*

In [10]:
# Create our portfolios from the loaded CSV data

# Create a scope for our portfolio
scope = 'portfolio_demo'

# Set the code of your portfolio
portfolio_code = 'Global-Strategies'

# Set the creation date of your portfolio
effective_date = datetime(2018, 1, 1, tzinfo=pytz.utc)
created_portfolios = []

# Next we create a request for the portfolio
request = models.CreateTransactionPortfolioRequest(
    code=portfolio_code,
    display_name="Global Strategies Fund",
    base_currency="EUR",
    created=effective_date,
    description=None,
    corporate_action_source_id=None,
    accounting_method=None,
    sub_holding_keys=None,
    properties=None)
# And finally we can upsert the portfolio creation request to LUSID
result = client.transaction_portfolios.create_portfolio(
    scope=scope,
    create_request=request)
# Save the portfolio to a list for easy access later on
created_portfolios.append(result)

# Pretty print the response
prettyprint.portfolio_response(result)

Portfolio Created
Scope: portfolio_demo
Code: Global-Strategies
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2019-08-12 10:40:44.702221+00:00



Now we are going to add some more cut labels for use in this example - one for the beginning of the day, *"LondonOpen"*, and one for the middle of the day, *"LondonLunch"*. For simplicity we are going to use one timezone, however in practice multiple can be used. <br/>
We can then use these to examine the bi-temporal features of LUSID with cut labels. 

Read more about bi-temporal data here: [LUSID Knowledge base: Bi-Temporal Data](https://support.lusid.com/what-is-bi-temporal-data)

*Run the cell below to create these cut labels*

In [11]:
### Create LondonOpen cut label ###
# Create the time for the cut label
londonTimeOpen = models.CutLocalTime(hours=9, minutes=0)

# Define the parameters of the cut label in a request
request = models.CutLabelDefinition(
    code="LondonOpen", 
    description="London Opening Time, 9am in UK", 
    display_name="LondonOpen",
    cut_local_time=londonTimeOpen,
    time_zone="GB")

# Upsert the request to LUSID to create the cut label
result = client.cut_labels.create_cut_label_definition(
    create_request=request)

prettyprint.cut_label_response(result)

### Create LondonLunch cut label ###
# Create the time for the cut label
londonTimeLunch = models.CutLocalTime(hours=12, minutes=0)

# Define the parameters of the cut label in a request
request = models.CutLabelDefinition(
    code="LondonLunch", 
    description="London Lunch Time, 12pm in UK", 
    display_name="LondonLunch",
    cut_local_time=londonTimeLunch,
    time_zone="GB")

# Upsert the request to LUSID to create the cut label
result = client.cut_labels.create_cut_label_definition(
    create_request=request)

prettyprint.cut_label_response(result)

Cut Label Created
Display Name: LondonOpen
Code: LondonOpen
Local Time: 9:00
Timezone: GB
Description: London Opening Time, 9am in UK

Cut Label Created
Display Name: LondonLunch
Code: LondonLunch
Local Time: 12:00
Timezone: GB
Description: London Lunch Time, 12pm in UK



We will now also define a function to write our cut label onto a date in a format readable to LUSID

> Date-times are usually in the form "2018-08-31T07:36:53.523415+01:00" 
>
>Cut labels are written in the form "2018-08-31NSingaporeClose"

In [12]:
# Defines a function to write the date and cut label in the format "YYYY-MM-DDN{cut_label_code}""
def cut_label_writer(date, cut_label_code):
    return str(date) + "N" + cut_label_code

## 3. Updating our Portfolio
If you notice above we used EUR as the base currency in our portfolio - we now want to change this. Therefore we will edit our portfolio by upserting updated portfolio details into LUSID. This can also be used to insert more details into our portfolio, or to change other details of our portfolio.

Read more about the behaviour of the upsert method here: [LUSID Knowledge Base: Upsert Method](https://support.lusid.com/upsert-command) <br/>
Also see here for more details on upserting portfolio details [LUSID API Docs: Upsert Portfolio Details](https://docs.lusid.com/#operation/UpsertPortfolioDetails)

We will use our LondonLunch cut label to make this change to our portfolio, and then examine the portfolio before and after the change using our other cut labels and LUSID's bi-temporal features to confirm that the change has been made.

*Run the cell below to upsert our corrected details*

In [13]:
# Create the change to our portfolio
request = models.CreatePortfolioDetails(
    base_currency="GBP")

response = client.transaction_portfolios.upsert_portfolio_details(
    scope=scope,
    code=portfolio_code,
    effective_at=cut_label_writer(date.today(), "LondonLunch"),
    details=request)

In [14]:
# Get portfolio before change using our cut labels
time_before = cut_label_writer(date.today(), "LondonOpen")

response = client.transaction_portfolios.get_details(
    scope=scope,
    code=portfolio_code,
    effective_at=time_before)

prettyprint.portfolio_details_response(response)

# Get portfolio after change using our cut labels
time_after = cut_label_writer(date.today(), "LondonClose")

response = client.transaction_portfolios.get_details(
    scope=scope,
    code=portfolio_code,
    effective_at=time_after)

prettyprint.portfolio_details_response(response)

Portfolio Details: 
Detail Origin Portfolio Scope: portfolio_demo
Detail Origin Portfolio Code: Global-Strategies
Base Currency: EUR
Corporate Action Source Id: None


Portfolio Details: 
Detail Origin Portfolio Scope: portfolio_demo
Detail Origin Portfolio Code: Global-Strategies
Base Currency: GBP
Corporate Action Source Id: None




## 4. Add your Instrument Universe
We now want to make trades and take on holdings using our cut labels, but before we can do this we need to populate our instrument universe. In this case we are importing our instrument universe from a CSV file. 

Read more about instruments in LUSID in the [LUSID Knowledge Base: Instruments](https://support.lusid.com/what-is-an-instrument).

*Run the cell below to import your instrument universe*

In [15]:
instrument_universe = pd.read_csv('data/instruments.csv')
instrument_universe.head()

,instrument_name,client_internal,currency,isin,figi,exchange_code,country_issue,ticker,market_sector,security_type,coupon
0,Amazon_Nasdaq_AMZN,imd_34634534,USD,US0231351067,BBG000BVPXP1,UN,united_states_america,AMZN,equity,common_stock,NaN
1,Apple_Nasdaq_AAPL,imd_35345345,USD,US0378331005,BBG000B9XVV8,UN,united_states_america,AAPL,equity,common_stock,NaN
2,BP_LondonStockEx_BP,imd_43535553,GBP,GB0007980591,BBG000C05BD1,LN,united_kingdom,BP/,equity,common_stock,NaN
3,BurfordCapital_LondonStockEx_BUR,imd_43534356,GBP,GG00B4L84979,BBG000PN88Q7,LN,united_kingdom,BUR,equity,common_stock,NaN
4,EKFDiagnostics_LondonStockEx_EKF,imd_34535355,GBP,GB0031509804,BBG000BVNBN3,LN,united_kingdom,EKF,equity,common_stock,NaN


Now that you have the details for your instruments you can go ahead and create an instrument definition for each instrument. These can then be upserted into LUSID. Read about instrument definitions here [LUSID Knowledge Base: What is an Instrument?](https://support.lusid.com/what-is-an-instrument).

For further usage of the upsert instruments API call refer to the [LUSID API Docs: Upserting Instruments](https://docs.lusid.com/#operation/UpsertInstruments).

*Run the cell below to upsert your instruments into LUSID*

In [16]:
# Initialise your batch upsert request
batch_upsert_request = {}

# Using your instrument universe create your batch request
for index, instrument in instrument_universe.iterrows():

    # Specify the columns of your identifiers
    identifier_columns = [
            ('isin', 'Isin'), 
            ('figi', 'Figi'), 
            ('ticker', 'Ticker'),
            ('client_internal', 'ClientInternal')
    ]
    
    # Create your identifiers
    identifiers = {}
    for identifier in identifier_columns:
        identifiers[identifier[1]] = models.InstrumentIdValue(
            value=instrument[identifier[0]])
    
    # Build your request and add it to the dictionary
    batch_upsert_request[instrument['instrument_name']] = models.InstrumentDefinition(
        name=instrument['instrument_name'],
        identifiers=identifiers)
    
# Call LUSID to upsert your instrument defintions
instrument_response = client.instruments.upsert_instruments(
    requests=batch_upsert_request)

# Pretty print the response
prettyprint.instrument_response(instrument_response)

Instrument Successfully Upserted: Sage_LondonStockEx_SGE
ClientInternal ID: imd_23536673
LUSID Instrument ID: LUID_DVHS1611


Instrument Successfully Upserted: USTreasury_6.875_2025
ClientInternal ID: imd_34534539
LUSID Instrument ID: LUID_IFWAHLX6


Instrument Successfully Upserted: UKGiltTreasury_3.75_2021
ClientInternal ID: imd_34643653
LUSID Instrument ID: LUID_3JUBXF9T


Instrument Successfully Upserted: Whitebread_LondonStockEx_WTB
ClientInternal ID: imd_35349900
LUSID Instrument ID: LUID_OEFL7HSH


Instrument Successfully Upserted: BurfordCapital_LondonStockEx_BUR
ClientInternal ID: imd_43534356
LUSID Instrument ID: LUID_FQOH3BIM


Instrument Successfully Upserted: WPP_LondonStockEx_WPP
ClientInternal ID: imd_34536734
LUSID Instrument ID: LUID_N601RM6G


Instrument Successfully Upserted: Kingfisher_LondonStockEx_KGF
ClientInternal ID: imd_34535552
LUSID Instrument ID: LUID_68SFRG3A


Instrument Successfully Upserted: JustEat_LondonStockEx_JE
ClientInternal ID: imd_35436366
LUSID

## 5. Set your Initial Holdings
Now that you have your instrument universe populated you can load your initial holdings into your portfolio. In this case you will import your holdings from a CSV file. 

*Run the cell below to import your take on balances*

In [17]:
holdings = pd.read_csv('data/demo-holdings.csv')
holdings.head()

,portfolio_code,instrument_name,quantity,price,currency,figi
0,Global-Strategies,GBP_Cash,8336000,1.000,GBP,NaN
1,Global-Strategies,Glencore_LondonStockEx_GLEN,905141,2.762,GBP,BBG001MM1KV4
2,Global-Strategies,Kingfisher_LondonStockEx_KGF,1362038,2.276,GBP,BBG000BKH1W6
3,Global-Strategies,UKGiltTreasury_2.0_2025,405589,106.637,GBP,BBG0088JSC32
4,Global-Strategies,UKGiltTreasury_3.5_2045,266169,134.433,GBP,BBG006N6HZM7


Now that you have imported your holdings you can add them to LUSID. You can do this by setting the holdings on a portfolio. 

Read more about how holdings are generated in LUSID here [LUSID Knowledge Base: The effect of holding adjustments](https://support.lusid.com/how-are-holdings-generate-by-lusid).

First we will need to set an effective date from which our holdings are recorded, which we will set as five days ago from the start of the day using our *"LondonOpen"* cut label.

For further usage of the set holdings API call refer to the [LUSID API Docs: Set Holdings](https://docs.lusid.com/#operation/SetHoldings).

*Run the cell below to set our effective date and upsert your holdings into LUSID*

In [18]:
# Make the holdings effective date
holdings_effective_date = date.today() - timedelta(days=5)
holdings_cut_label_code = "LondonOpen"

holdings_effective_cut_label = cut_label_writer(holdings_effective_date, holdings_cut_label_code)

print("Effective date: " + holdings_effective_cut_label)

# Iterate the portfolios in the holdings CSV, note in this case you only have one
for portfolio in holdings['portfolio_code'].unique():
    # Initialise a list to hold your adjustments
    holding_adjustments = []
    
    # Iterate over the holdings in each portfolio
    for index, holding in holdings.loc[holdings['portfolio_code'] == portfolio].iterrows():
        
        # Set your instrument identifiers based on whether or not instrument is cash
        if 'Cash' in holding['instrument_name']:
            identifier_key = 'Instrument/default/Currency'
            identifer = holding['instrument_name'].split('_')[0]
        else:
            identifier_key = 'Instrument/default/Figi'
            identifer = holding['figi']
            
        # Create your holding adjustment and append it to your list
        holding_adjustments.append(
            models.AdjustHoldingRequest(
                instrument_identifiers={
                    identifier_key: identifer},
                tax_lots=[
                    models.TargetTaxLotRequest(
                        units=holding['quantity'],
                        cost=models.CurrencyAndAmount(
                            amount=holding['quantity'] * holding['price'],
                            currency=holding['currency']),
                        portfolio_cost=holding['quantity'] * holding['price'],
                        price=holding['price'])
                ]
            )
        )
    
    # Call LUSID to set your initial holdings
    response = client.transaction_portfolios.set_holdings(
        scope=scope,
        code=portfolio,
        effective_at=holdings_effective_cut_label,
        holding_adjustments=holding_adjustments)

    # Pretty print our response from LUSID
    prettyprint.set_holdings_response(response, scope, portfolio)

Effective date: 2019-08-07NLondonOpen
Holdings Successfully Set for Portfolio
Scope: portfolio_demo
Code: Global-Strategies
Holdings Effective From: 2019-08-12 11:00:00+00:00
Holdings Created On: 2019-08-12 10:40:47.590594+00:00



## 6. Get Output Transactions
You can see how LUSID has adjusted the holdings of our portfolio to meet our initial holdings by generating the output transactions from the portfolio. You can read more about output transactions here [LUSID Knowledge Base: Output Transactions](https://support.lusid.com/what-is-an-output-transaction-in-lusid).

To build these transactions we need to set a start date and an end date - again we will use our cut labels (LondonOpen and LondonClose) for these, and we will set our start and end dates to go with this cut label as 4 days either side of the effective date of our holdings.

For further usage of the build transactions API call refer to the [LUSID API Docs: Build Output Transactions](https://docs.lusid.com/#operation/BuildTransactions).

*Run the cell below to generate your output transactions*

In [19]:
# Set start date cut label
start_date_label = cut_label_writer(
        holdings_effective_date-timedelta(days=4),
        "LondonOpen")
# Set end date cut label
end_date_label = end_date=cut_label_writer(
        holdings_effective_date+timedelta(days=4),
        "LondonClose")

print("start date: " + start_date_label)
print("end date: " + end_date_label + "/n")

# Set our query parameters to build your transactions
query_params = models.TransactionQueryParameters(
    start_date=start_date_label,
    end_date=end_date_label,
    query_mode='TradeDate',
    show_cancelled_transactions=None)

# Call LUSID to build your output transactions
response = client.transaction_portfolios.build_transactions(
    scope=scope,
    code=portfolio_code,
    property_keys=['Instrument/default/Name'],
    parameters=query_params
)

# Pretty print your output transactions
prettyprint.output_transactions(response, scope, portfolio_code, ['Instrument/default/Name'])

start date: 2019-08-03NLondonOpen
end date: 2019-08-11NLondonClose/n
Output Transactions for Portfolio
Scope: portfolio_demo
Code: Global-Strategies

Transaction Id: 2019-08-07T08:00:00.0000000+00:00
Transaction Type: AdjustmentIncrease
Instrument/default/Name: Glencore_LondonStockEx_GLEN
Units: 905141.0
Price: 2.762
Currency: GBP
Transaction Date: 2019-08-07 08:00:00+00:00
Settlement Date: 2019-08-07 08:00:00+00:00

Transaction Id: 2019-08-07T08:00:00.0000000+00:00
Transaction Type: AdjustmentIncrease
Instrument/default/Name: Kingfisher_LondonStockEx_KGF
Units: 1362038.0
Price: 2.276
Currency: GBP
Transaction Date: 2019-08-07 08:00:00+00:00
Settlement Date: 2019-08-07 08:00:00+00:00

Transaction Id: 2019-08-07T08:00:00.0000000+00:00
Transaction Type: AdjustmentIncrease
Instrument/default/Name: UKGiltTreasury_2.0_2025
Units: 405589.0
Price: 106.637
Currency: GBP
Transaction Date: 2019-08-07 08:00:00+00:00
Settlement Date: 2019-08-07 08:00:00+00:00

Transaction Id: 2019-08-07T08:00:00.0

## 7. Add your Transactions Throughout the Day
Now that you have set your initial holdings you are ready to take on some transactions. You have made several trades over the last day that you would like to add to LUSID. You will import these from a CSV file.

*Run the cell below to import your transactions*

In [20]:
daily_transactions = pd.read_csv('data/demo-transactions-2.csv')
daily_transactions.head()

,portfolio_code,transaction_id,instrument_name,transaction_description,transaction_type,transaction_units,transaction_price,transaction_currency,transaction_strategy,transaction_cost,figi
0,Global-Strategies,tid_329432525234324,Kingfisher_LondonStockEx_KGF,Equity Sale,Sell,325000,2.345,GBP,quantitativeSignal,762125.000,BBG000BKH1W6
1,Global-Strategies,tid_325452342424500,UKGiltTreasury_4.5_2034,Equity Purchase,Buy,10501,140.572,GBP,incomeRequirements,1476146.572,BBG0000D14P3
2,Global-Strategies,tid_234295929052090,UKGiltTreasury_3.75_2021,Equity Purchase,Buy,24000,109.126,GBP,incomeRequirements,2619024.000,BBG001KKJLR4
3,Global-Strategies,tid_234942982496001,USTreasury_2.00_2021,Equity Sale,Sell,57000,97.800,USD,internationalExposure,5574600.000,BBG00FN3B5K8
4,Global-Strategies,tid_121309590059995,BP_LondonStockEx_BP,Equity Purchase,Buy,50000,5.287,GBP,fundamentalAnalysis,264350.000,BBG000C05BD1


Now that we have imported our transactions we can upsert them into LUSID. We will again use our cut labels to set our transaction and settlement dates.

Read more about transactions here [LUSID Knowledge Base: Transactions](https://support.lusid.com/what-is-a-transaction). 

For further usage of the upsert transactions API call refer to the [LUSID API Docs: Upsert Transactions](https://docs.lusid.com/#operation/UpsertTransactions).

*Run the cell below to upsert our transactions into LUSID*

In [21]:
# Set your transaction and settlement dates
transaction_date = date.today()-timedelta(days=1)
settlement_date = date.today()+timedelta(days=1)

# Turn them into cut labels
transaction_cut_label = cut_label_writer(transaction_date, "LondonClose")
settlement_cut_label = cut_label_writer(settlement_date, "LondonOpen")

print("Transaction date: " + transaction_cut_label)
print("Settlement date: " + settlement_cut_label)

# Iterate over the portfolios in our transactions file, in this case only one
for portfolio in daily_transactions['portfolio_code'].unique():
    
    # Initialise a list to hold your transaction requests
    transactions = []
    
    # Iterate over the transactions in the portfolio
    for index, transaction in daily_transactions.loc[daily_transactions['portfolio_code'] == portfolio].iterrows():
        
        # Append your request to the list
        transactions.append(models.TransactionRequest(
            transaction_id=transaction['transaction_id'],
            type=transaction['transaction_type'],
            instrument_identifiers={
                'Instrument/default/Figi': transaction['figi']
            },
            transaction_date=transaction_cut_label,
            settlement_date=settlement_cut_label,
            units=transaction['transaction_units'],
            transaction_price=models.TransactionPrice(
                  price=transaction['transaction_price'],
                  type='Price'),
            total_consideration=models.CurrencyAndAmount(
              amount=transaction['transaction_cost'],
              currency=transaction['transaction_currency']),
             source='Default',
             transaction_currency=transaction['transaction_currency']))
        
    # Call LUSID to upsert your transactions for this portfolio
    response = client.transaction_portfolios.upsert_transactions(
        scope=scope,
        code=portfolio,
        transactions=transactions)
    
    # Print the response from LUSID using pretty formatting 
    prettyprint.transactions_response(
        response,
        scope,
        portfolio)

Transaction date: 2019-08-11NLondonClose
Settlement date: 2019-08-13NLondonOpen
Transactions Successfully Upserted into Portfolio
Scope: portfolio_demo
Code: Global-Strategies
Transactions Effective From: 2019-08-11 16:30:00+00:00
Transactions Created On: 2019-08-12 10:40:48.542722+00:00



## 8. Retrieve your Transactions
Now we can go back and look at the transactions now countained in our portfolio, and we can get these transactions over a given interval of effective time, which we will use our cut labels for in this example. 

As we have only upserted our transactions once, we will use the same cut label for both our "from transaction date" and "to transacion date", however with multiple transactions we can use a range of cut labels to filter our results.

For further usage of the get transactions API call refer to the [LUSID API Docs: Get Transactions](https://docs.lusid.com/#operation/GetTransactions).
                                                                                                   
*Run the cell below to get your transactions*                                                                                                   

In [22]:
# Call LUSID to get your transactions made between these cut labels
response = client.transaction_portfolios.get_transactions(
    scope=scope,
    code=portfolio_code,
    from_transaction_date=transaction_cut_label, 
    to_transaction_date=transaction_cut_label)

# Pretty print the response
prettyprint.get_transactions_response(
    response,
    scope,
    portfolio_code,
    [])

Transactions Retrieved from Portfolio
Scope:  portfolio_demo
Code:  Global-Strategies 

Transaction Id: tid_329432525234324
Transaction Type: Sell
Transaction/default/SourcePortfolioId: Global-Strategies
Transaction/default/SourcePortfolioScope: portfolio_demo
Units: 325000.0
Price: 2.345
Currency: GBP
Transaction Date: 2019-08-11 16:30:00+00:00

Transaction Id: tid_325452342424500
Transaction Type: Buy
Transaction/default/SourcePortfolioId: Global-Strategies
Transaction/default/SourcePortfolioScope: portfolio_demo
Units: 10501.0
Price: 140.572
Currency: GBP
Transaction Date: 2019-08-11 16:30:00+00:00

Transaction Id: tid_234295929052090
Transaction Type: Buy
Transaction/default/SourcePortfolioId: Global-Strategies
Transaction/default/SourcePortfolioScope: portfolio_demo
Units: 24000.0
Price: 109.126
Currency: GBP
Transaction Date: 2019-08-11 16:30:00+00:00

Transaction Id: tid_234942982496001
Transaction Type: Sell
Transaction/default/SourcePortfolioId: Global-Strategies
Transaction/d

## 9. Retrieve your Holdings
You can see how these transactions have affected your holdings by trying to retrieve the holdings of your portfolio. We will use our cut labels to examine our holdings before and after our transactions were upserted.

For further usage of the get holdings API call refer to the [LUSID API Docs: Get Holdings](https://docs.lusid.com/#operation/GetHoldings).

*Run the cells below to get our holdings from our portfolio before and after upserting our transactions*

In [23]:
# Create a cut label string for before the transaction occurred
before_transaction_cut_label = cut_label_writer(transaction_date, "LondonOpen")

# Get holdings before transaction 
result = client.transaction_portfolios.get_holdings(
    scope=scope,
    code=portfolio_code,
    property_keys=['Instrument/default/Name'],
    effective_at=before_transaction_cut_label)

print("Effective at: " + before_transaction_cut_label)
prettyprint.holdings_response(result, scope, portfolio_code)

Effective at: 2019-08-11NLondonOpen
Holdings for Portfolio
Scope: portfolio_demo
Code: Global-Strategies

Instrument/default/Name: Glencore_LondonStockEx_GLEN
Units: 905141.0
Cost: 2499999.44
Currency: GBP


Instrument/default/Name: Kingfisher_LondonStockEx_KGF
Units: 1362038.0
Cost: 3099998.49
Currency: GBP


Instrument/default/Name: UKGiltTreasury_2.0_2025
Units: 405589.0
Cost: 43250794.19
Currency: GBP


Instrument/default/Name: UKGiltTreasury_3.5_2045
Units: 266169.0
Cost: 35781897.18
Currency: GBP


Instrument/default/Name: UKGiltTreasury_3.75_2021
Units: 661713.0
Cost: 71548379.84
Currency: GBP


Instrument/default/Name: UKGiltTreasury_4.5_2034
Units: 77481.0
Cost: 10891659.13
Currency: GBP


Instrument/default/Name: USTreasury_2.00_2021
Units: 1440244.0
Cost: 140999887.6
Currency: USD


Instrument/default/Name: BurfordCapital_LondonStockEx_BUR
Units: 853486.0
Cost: 12000013.16
Currency: GBP


Instrument/default/Name: EKFDiagnostics_LondonStockEx_EKF
Units: 925925.0
Cost: 249999.

In [24]:
# Get holdings after transaction    
result = client.transaction_portfolios.get_holdings(
    scope=scope,
    code=portfolio_code,
    property_keys=['Instrument/default/Name'],
    effective_at=transaction_cut_label)

print("Effective at: " + transaction_cut_label)
prettyprint.holdings_response(result, scope, portfolio_code)

Effective at: 2019-08-11NLondonClose
Holdings for Portfolio
Scope: portfolio_demo
Code: Global-Strategies

Instrument/default/Name: Glencore_LondonStockEx_GLEN
Units: 905141.0
Cost: 2499999.44
Currency: GBP


Instrument/default/Name: Kingfisher_LondonStockEx_KGF
Units: 1037038.0
Cost: 2360298.49
Currency: GBP


Instrument/default/Name: UKGiltTreasury_2.0_2025
Units: 405589.0
Cost: 43250794.19
Currency: GBP


Instrument/default/Name: UKGiltTreasury_3.5_2045
Units: 266169.0
Cost: 35781897.18
Currency: GBP


Instrument/default/Name: UKGiltTreasury_3.75_2021
Units: 685713.0
Cost: 74167403.84
Currency: GBP


Instrument/default/Name: UKGiltTreasury_4.5_2034
Units: 87982.0
Cost: 12367805.7
Currency: GBP


Instrument/default/Name: USTreasury_2.00_2021
Units: 1411244.0
Cost: 138166387.6
Currency: USD


Instrument/default/Name: BurfordCapital_LondonStockEx_BUR
Units: 853486.0
Cost: 12000013.16
Currency: GBP


Instrument/default/Name: EKFDiagnostics_LondonStockEx_EKF
Units: 925925.0
Cost: 249999.

## 10. Adjust Holdings with your End of Day Position
Great, you are able to generate your holdings from LUSID! Now that you have succesfully added your transactions into LUSID, you can update your portfolio with the end of day position to ensure that your source system and LUSID are in sync.

You will import your end of day holdings from a CSV file.

*Run the cell below to import your end of day holdings*

In [25]:
end_of_day_holdings = pd.read_csv('data/demo-holdings2.csv')
end_of_day_holdings.head()

,portfolio_code,instrument_name,quantity,price,currency,figi
0,Global-Strategies,GBP_Cash,8336000,1.000,GBP,NaN
1,Global-Strategies,Glencore_LondonStockEx_GLEN,905141,2.762,GBP,BBG001MM1KV4
2,Global-Strategies,Kingfisher_LondonStockEx_KGF,1037038,2.276,GBP,BBG000BKH1W6
3,Global-Strategies,UKGiltTreasury_2.0_2025,405589,106.637,GBP,BBG0088JSC32
4,Global-Strategies,UKGiltTreasury_3.5_2045,266169,134.433,GBP,BBG006N6HZM7


Now that you have your holdings you can set your portfolio holdings just like you did at the start of the day to ensure that it matches the source system perfectly. Again, we will use a cut label, this time *"LondonLunch"* in order to set the effective date of our holdings.

*Run the cell below to adjust your holdings to these levels*

In [26]:
# Set the holdings to be effective from yesterday lunchtime with a cut label
holdings_effective_date = date.today() - timedelta(days=1)
holdings_cut_label_code = "LondonLunch"

holdings_effective_cut_label = cut_label_writer(holdings_effective_date, holdings_cut_label_code)

# Print out the details for the cut label we are using
details = client.cut_labels.get_cut_label_definition(
    code="LondonLunch")
prettyprint.get_cut_label(details)

# Iterate over the portfolios in our holdings, in this case only one
for portfolio in end_of_day_holdings['portfolio_code'].unique():
    
    # Initialise a list to hold your adjustments
    holding_adjustments = []
    
    # Iterate over the end of day holdings for this portfolio
    for index, holding in end_of_day_holdings.loc[end_of_day_holdings['portfolio_code'] == portfolio].iterrows():
        
        # Ensure that you use the correct identifier based on the instrument
        if 'Cash' in holding['instrument_name']:
            identifier_key = 'Instrument/default/Currency'
            identifer = holding['instrument_name'].split('_')[0]
        else:
            identifier_key = 'Instrument/default/Figi'
            identifer = holding['figi']
            
        # Build your adjustment and add it to your list
        holding_adjustments.append(
            models.AdjustHoldingRequest(
                instrument_identifiers={
                    identifier_key: identifer},
                tax_lots=[
                    models.TargetTaxLotRequest(
                        units=holding['quantity'],
                        cost=models.CurrencyAndAmount(
                            amount=holding['quantity'] * holding['price'],
                            currency=holding['currency']),
                        portfolio_cost=holding['quantity'] * holding['price'],
                        price=holding['price'])
                ]
            )
        )
        
    # Call LUSID to set your end of day position
    response = client.transaction_portfolios.adjust_holdings(
        scope=scope,
        code=portfolio,
        effective_at=holdings_effective_cut_label, 
        holding_adjustments=holding_adjustments)

    # Pretty print the response from LUSID
    prettyprint.adjust_holdings_response(
        response, 
        scope, 
        portfolio)

Cut Label Details:
Display Name: LondonLunch
Code: LondonLunch
Local Time: 12:00
Timezone: GB
Description: London Lunch Time, 12pm in UK

Holdings Successfully Adjusted for Portfolio
Scope: portfolio_demo
Code: Global-Strategies
Adjusted Holdings Effective From: 2019-08-12 11:00:00+00:00
Adjusted Holdings Created On: 2019-08-12 10:40:49.998807+00:00



# *<center>~Starts getting a bit iffy from here~</center>*
- tough to use cut labels on get holdings adjustment
- ok to use them on cancelling but not ideal
- but also no other examples of list, get, or cancel holdings adjustments? would here be a good place to have them?

## 11. Cancelling Holdings Adjustments
If we then decide we want to cancel the adjustment of our holdings, we can! We will now cancel the adjustment we just made by specifying the scope and code of our portfolio, and the effectiveAt time of our holdings adjustment, where we will use the same cut label we used for the initial adjustment.

>Cancelling adjustments is ideal to see the effects that these previous adjustments had to the current state of our portfolio, and we can do this using the derived portfolio feature in LUSID. <br/>
>
>For simplicity of this example and to emphasise the use of cut labels, we are just going to cancel our adjustments on our original portfolio, however you can explore derived portfolios further in our other use cases.
>
>Read more about derived portfolios here [LUSID Knowledge Base: Derived Portfolios](https://support.lusid.com/what-is-a-derived-portfolio)

*Run the cell below to cancel the adjustment to your holdings*

### 11.1. List All Holding Adjustments Before Cancellation
First, we will list all of the adjustments we have made to our holdings.

As with retrieving our transactions, we can choose to look at adjustments over a range of dates or cut labels, so we will use the start and end date cut labels we used to get our output transactions to look at our adjustments as well.

The *"Unmatched Holding Method"* we get as a response describes how we adjusted our holdings. <br/>
*"PositionToZero"* describes a call to *"Set holdings"* where the entire transaction portfolio's holdings were set, whereas *"KeepTheSame"* describes a call to *"Adjust holdings"* where only specified holdings were adjusted.

*Run the cell below to list your holdings adjustments*

In [27]:
print("From effective at: " + start_date_label)
print("To effective at: " + end_date_label + "\n")

# list holding adjustments
response = client.transaction_portfolios.list_holdings_adjustments(
    scope=scope,
    code=portfolio_code,
    from_effective_at=start_date_label,
    to_effective_at=end_date_label) 

prettyprint.list_holdings_adjustments_response(
    response,
    scope,
    portfolio)

From effective at: 2019-08-03NLondonOpen
To effective at: 2019-08-11NLondonClose

Holding Adjustments for Portfolio:
Scope: portfolio_demo
Code: Global-Strategies

Unmatched Holding Method: PositionToZero
Adjustment Effective From: 2019-08-07 08:00:00+00:00
Adjustment Created On: 2019-08-12 10:40:47.590594+00:00

Unmatched Holding Method: KeepTheSame
Adjustment Effective From: 2019-08-11 11:00:00+00:00
Adjustment Created On: 2019-08-12 10:40:49.998807+00:00



### 11.2. Get the details of these adjustments
From the above list of holdings adjustments, we can then examine them further using their *"Adjustment Effective From"* datetime and the get holdings adjustment API call: [LUSID API Docs: Get Holdings Adjustment](https://docs.lusid.com/api#operation/GetHoldingsAdjustment)

We can see that the time our adjustments are effective from coincides with 

*Run the cell below to get the adjustment details for the above "KeepTheSame" adjustment*

In [30]:
print(holdings_effective_cut_label)

response = client.transaction_portfolios.get_holdings_adjustment(
    scope=scope,
    code=portfolio_code, 
    effective_at="2019-08-11 11:00:00+00:00") #holdings_effective_cut_label

prettyprint.get_holdings_adjustment_resonse(
    response,
    scope,
    portfolio)

2019-08-11NLondonLunch


ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Mon, 12 Aug 2019 10:43:13 GMT', 'Content-Type': 'application/problem+json; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'lusid-meta-success': 'False', 'lusid-meta-duration': '1', 'lusid-meta-requestId': '0HLOTPJEJE3GK:00000001', 'Server': 'FINBOURNE', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains'})
HTTP response body: {"name":"InvalidParameterValue","errorDetails":[],"code":151,"type":"https://docs.lusid.com/#section/Error-Codes/151","title":"Date 2019-08-11 11:00:00+00:00 must be formatted as an ISO date, eg 'yyyy-MM-ddTHH:mm:ss.FFFFFFFK'","status":400,"detail":"Date 2019-08-11 11:00:00+00:00 must be formatted as an ISO date, eg 'yyyy-MM-ddTHH:mm:ss.FFFFFFFK'","instance":"lusid:fbn-prd.lusid.com/0HLOTPJEJE3GK/00000001"}


### 11.3. Cancel our adjustment 
We will now cancel the adjustment we just made by specifying the scope and code of our portfolio, and the effectiveAt time of our holdings adjustment, where we will use the same cut label we used for the initial adjustment.

For detailed usage of the cancel adjust holdings API call refer to the [LUSID API Docs: Cancel Adjust Holdings](https://docs.lusid.com/api#operation/CancelAdjustHoldings)

*Run the cell below to cancel our adjustment*

In [31]:
response = client.transaction_portfolios.cancel_adjust_holdings(
    scope=scope,
    code=portfolio_code,
    effective_at=holdings_effective_cut_label)

prettyprint.cancel_adjust_holdings_response(
    response,
    scope,
    portfolio)

Holdings Adjustment Cancelled for Portfolio
Scope: portfolio_demo
Code: Global-Strategies
Adjusted Holdings Effective From: 2019-08-12 11:00:00+00:00
Adjusted Holdings Created On: 2019-08-12 10:43:14.513198+00:00



### 11.4 List holdings adjustments after cancellation
Now we will list our holdings adjustments once more to check that they have changed - you should see that the "KeepTheSame" adjustment that we had before has been deleted. Note that this time we haven't specified any range of dates, so this will list all of our adjustments.

*Run the cell below to list our holdings adjustments*

In [32]:
# list holding adjustments
response = client.transaction_portfolios.list_holdings_adjustments(
    scope=scope,
    code=portfolio_code) 

prettyprint.list_holdings_adjustments_response(
    response,
    scope,
    portfolio)

Holding Adjustments for Portfolio:
Scope: portfolio_demo
Code: Global-Strategies

Unmatched Holding Method: PositionToZero
Adjustment Effective From: 2019-08-07 08:00:00+00:00
Adjustment Created On: 2019-08-12 10:40:47.590594+00:00



## 12. Delete Cut Labels and Portfolios
Finally, we will delete all of our existing cut labels and our portfolio, in order to tidy up our LUSID environment from this example and allowing it to be run again in its entirety.

*Run the cells below to clear up your LUSID environment*

In [33]:
# Deletes all cut labels
for body in client.cut_labels.list_cut_label_definitions().values:
    client.cut_labels.delete_cut_label_definition(
        code=body.code)

# Check they've been deleted
prettyprint.list_cut_labels(client.cut_labels.list_cut_label_definitions())

Existing Cut Labels:


In [34]:
# need to write code to delete all portfolios, trades etc. at end

# Delete portfolio
delete = client.portfolios.delete_portfolio(
    scope=scope, 
    code=portfolio_code)